In [ ]:
# This example needs the following packages: earthsim and quest
from holoviews.streams import PointDraw, PolyEdit, BoxEdit, PolyDraw
from requests.packages.urllib3.util.retry import Retry
from IPython.display import display, clear_output
from ipywidgets import widgets, HBox, Label
from requests.adapters import HTTPAdapter
from ipywidgets import widgets, Layout
from rasterio.io import MemoryFile
from itertools import product
from paramnb import Widgets
from io import BytesIO
from PIL import Image
import cartopy.crs as ccrs
import holoviews as hv
import geoviews as gv
import numpy as np
import requests
import rasterio
import warnings
import quest
import math
import sys
import os

hv.extension('bokeh')
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
# Helpful URLS:
    # https://stackoverflow.com/questions/15431044/can-i-set-max-retries-for-requests-request

In [ ]:
url_list = ['http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png',
            'https://s.basemaps.cartocdn.com/light_all/{Z}/{X}/{Y}.png',
            'http://tile.stamen.com/terrain/{Z}/{X}/{Y}.png',
            'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/World_Shaded_Relief/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/World_Physical_Map/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/USA_Topo_Maps/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/Ocean_Basemap/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{Z}/{Y}/{X}']

url_selection = widgets.Dropdown(value=url_list[0],options=url_list, description='Tile URL:', disabled=False, 
                              layout=Layout(width='75%', height='40px'))
display(url_selection)

zoom_widget_value = widgets.IntSlider(value=0,min=0,max=19,step=1,description='Zoom Level:',orientation='horizontal')
display(zoom_widget_value)

In [ ]:
url = url_selection.value
tiles = gv.WMTS(url, extents=(-180, -90, 180, 90), crs=ccrs.PlateCarree()).options(width=500, height=500)
boxes = gv.Polygons([], crs=ccrs.PlateCarree()).options(fill_alpha=0.4, color='blue', line_color='blue', line_width=2)
box_stream = BoxEdit(source=boxes)
tiles * boxes

In [ ]:
basemap_features = quest.api.get_features('svc://basemap:Web Mercator Tile Service')
collection_name = 'the_demo2'
if collection_name in quest.api.get_collections():
    pass
else:
    quest.api.new_collection(collection_name)

collection_feature = quest.api.add_features(collection_name, basemap_features[0])
collection_feature

In [ ]:
if box_stream.element:
    xs, ys = box_stream.element.array().T
    bbox = list(gv.util.project_extents((xs[0], ys[0], xs[2], ys[1]), ccrs.GOOGLE_MERCATOR, ccrs.PlateCarree()))
    print(bbox)
else:
    bbox = [-90.88, -32.35, -90.87, -32.36]
options = {'url': url_selection.value, 
           'zoom_level': zoom_widget_value.value, 
           'bbox': bbox, 
          }
staged_id = quest.api.stage_for_download(collection_feature, options=options)

In [ ]:
quest.api.download_datasets(staged_id)

In [ ]:
meta = quest.api.get_metadata(staged_id)

In [ ]:
meta